# Imports

In [1]:
 config = {
    'gdt': {
        'depth': 10,
        
        'learning_rate_index': 0.05,
        'learning_rate_values': 0.01,
        'learning_rate_leaf': 0.005,
        
        'split_index_activation': 'entmax',
                
        'optimizer': 'SWA', #adam with stochastic weight averaging
        
        'initializer': 'GlorotUniform', 
        
        'dropout': 0,
        
        'prunePostHoc': True,
        'prune_threshold': 1,
        
        'batch_size': 128,#512,
        'epochs': 10_000,
        
        'loss': 'binary_crossentropy', #binary_focal_crossentropy, binary_crossentropy
        
        'focalLossGamma': 3,
        
        'polyLoss': False,
        'polyLossEpsilon': 2,
        
        'restarts': 5,
        'restart_type': 'loss', #'loss', 'metric'
        
        'early_stopping_epochs': 200,
        'early_stopping_type': 'loss', #'loss', 'metric'
        'early_stopping_epsilon': 0.0,
    },
    
    'preprocessing': {
        'balance_threshold': 0.25,#.25, #if minclass fraction less than threshold/num_classes-1 | #0=no rebalance, 1=rebalance all
        'normalization_technique': 'quantile', #'min-max'
        'quantile_noise': 1e-3,
    },

    'computation': {
        'random_seed': 42,
        'trials': 10,
        
        'use_best_hpo_result': False,
        'hpo_path': '_DEFAULT',
        
        'force_depth': False,
        'force_dropout': False,
        'force_restart': True,   
        
        'use_gpu': True,
        'gpu_numbers': '0',#'1',
        'n_jobs': 10,
        'verbosity': 0,
        
        'hpo': None,
        'search_iterations': 300,
        'cv_num': 3,     
        
        'metrics_class': ['f1', 'roc_auc', 'accuracy', 'total_nodes','internal_node_num', 'leaf_node_num'],      
        'eval_metric_class': ['f1', 'roc_auc', 'accuracy', 'total_nodes', 'internal_node_num', 'leaf_node_num'], #f1 accuracy
    },
    
    'benchmarks': {
        'DNDT': True,
        'GeneticTree': True,
        'DL85': True,
        
        'sklearn': True,
        
        'NeuralNetwork': False,
    }
}

In [2]:
import numpy as np
np.set_printoptions(suppress=True)

import sklearn
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, ParameterGrid, ParameterSampler, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier, plot_tree, DecisionTreeRegressor
from sklearn.metrics import accuracy_score, f1_score, make_scorer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder, OrdinalEncoder
from pydl85 import DL85Classifier

from livelossplot import PlotLosses

import os
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt

from IPython.display import Image
from IPython.display import display, clear_output

import pandas as pd

if config['computation']['use_gpu']:
    os.environ['CUDA_VISIBLE_DEVICES'] = str(config['computation']['gpu_numbers'])
    os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
    os.environ['XLA_FLAGS'] = "--xla_gpu_cuda_data_dir=/usr/local/cuda-11.6"
    os.environ['TF_XLA_FLAGS'] = "--tf_xla_enable_xla_devices --tf_xla_auto_jit=2"    
else:
    os.environ['CUDA_VISIBLE_DEVICES'] = ''
    os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'false' 
#os.environ['TF_XLA_FLAGS'] = "--tf_xla_auto_jit=2 --tf_xla_cpu_global_jit" 


import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ["PYTHONWARNINGS"] = "ignore"
import logging

import tensorflow as tf
import tensorflow_addons as tfa

tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(3)

np.seterr(all="ignore")

from keras import backend as K

import seaborn as sns
sns.set_style("darkgrid")

import time
import random

from utilities.utilities_GDT import *
from utilities.GDT import *
from utilities.DNDT import *

from joblib import Parallel, delayed

from itertools import product
from collections.abc import Iterable

from copy import deepcopy
from pathlib import Path
import pickle
import dill

tf.random.set_seed(config['computation']['random_seed'])
np.random.seed(config['computation']['random_seed'])
random.seed(config['computation']['random_seed'])
tf.keras.utils.set_random_seed(config['computation']['random_seed'])

from datetime import datetime
timestr = datetime.utcnow().strftime('%Y-%m-%d--%H-%M-%S%f')
print(timestr)
os.makedirs(os.path.dirname("./evaluation_results" + config['computation']['hpo_path'] + "/latex_tables/" + timestr +"/"), exist_ok=True)

filepath = './evaluation_results' + config['computation']['hpo_path'] + '/depth' + str(config['gdt']['depth']) + '/' + timestr + '/'
Path(filepath).mkdir(parents=True, exist_ok=True)    

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

from joblib.externals.loky import set_loky_pickler
set_loky_pickler("dill")

2023-04-29--11-17-50446237


In [3]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num XLA-GPUs Available: ", len(tf.config.experimental.list_physical_devices('XLA_GPU')))

Num GPUs Available:  1
Num XLA-GPUs Available:  1


# Evaluation

## Real-World Eval

### Classification

In [4]:
identifier_list_classification_binary = [
                        'BIN:Blood Transfusion',# 748 4
                        'BIN:Banknote Authentication',# 1372 4
                        'BIN:Titanic',# 891 7 
                        'BIN:Raisins',#900 7
                        'BIN:Rice',#3810 7
                        'BIN:Echocardiogram',#132 8 ###TD
                        'BIN:Wisconsin Diagnostic Breast Cancer',# 569 10
                        'BIN:Loan House',# 614 11
                        'BIN:Heart Failure',# 299 12
                        'BIN:Heart Disease',# 303 13
                        'BIN:Adult',# 32561 14
                        'BIN:Bank Marketing',# 45211 14
                        'BIN:Congressional Voting',# 435, 16 ###TD
                        'BIN:Absenteeism',# 740 18
                        'BIN:Hepatitis',#155 19 ###TD
                        'BIN:German',# 1000 20
                        'BIN:Mushroom',#8124 22
                        'BIN:Credit Card',# 30000 23
                        'BIN:Horse Colic',#368 27
                        'BIN:Thyroid',#9172 29 ###TD
                        'BIN:Cervical Cancer',# 858 15
                        'BIN:Spambase',# 4601 57
                  ]       

In [5]:
benchmark_dict = get_benchmark_dict(config=config, eval_identifier='classification') 

parallel_eval_real_world = Parallel(n_jobs=min(config['computation']['n_jobs'], config['computation']['trials']), verbose=3, backend='loky') #loky #sequential multiprocessing
evaluation_results_real_world_classification_binary = parallel_eval_real_world(delayed(evaluate_real_world_parallel_nested)(identifier_list=identifier_list_classification_binary[::-1], 
                                                                                                                           random_seed_data=config['computation']['random_seed']+i,
                                                                                                                           random_seed_model=config['computation']['random_seed'],
                                                                                                                           config = config,
                                                                                                                           benchmark_dict = benchmark_dict,
                                                                                                                           metrics = config['computation']['metrics_class'],
                                                                                                                           verbosity = -1) for i in range(config['computation']['trials']))


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  5.9min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 ou

In [6]:
plot_table_save_results(benchmark_dict=benchmark_dict,
                        evaluation_results_real_world=evaluation_results_real_world_classification_binary,
                        identifier_list=identifier_list_classification_binary,                            
                        identifier_string='binary_test',
                        filepath=filepath,
                        config=config)      

plot_table_save_results(benchmark_dict=benchmark_dict,
                        evaluation_results_real_world=evaluation_results_real_world_classification_binary,
                        identifier_list=identifier_list_classification_binary,                            
                        identifier_string='binary_valid',
                        filepath=filepath,
                        config=config)  

plot_table_save_results(benchmark_dict=benchmark_dict,
                        evaluation_results_real_world=evaluation_results_real_world_classification_binary,
                        identifier_list=identifier_list_classification_binary,                            
                        identifier_string='binary_train',
                        filepath=filepath,
                        config=config)  

,GDT f1_mean,GDT f1_std,DNDT f1_mean,DNDT f1_std,GeneticTree f1_mean,GeneticTree f1_std,DL85 f1_mean,DL85 f1_std,sklearn f1_mean,sklearn f1_std
BIN:Blood Transfusion,0.627262,0.056431,0.558071,0.058859,0.574379,0.093202,0.589835,0.034122,0.573242,0.035920
BIN:Banknote Authentication,0.979913,0.007788,0.885838,0.023708,0.928084,0.022035,0.962230,0.010720,0.981429,0.006481
BIN:Titanic,0.781598,0.035472,0.740341,0.025999,0.762750,0.040826,0.753668,0.030818,0.734937,0.040876
BIN:Raisins,0.849657,0.024863,0.831822,0.025741,0.858937,0.021931,0.849137,0.026684,0.810901,0.027791
BIN:Rice,0.925941,0.006244,0.902482,0.017501,0.926751,0.004741,0.924574,0.008003,0.905246,0.010326
BIN:Echocardiogram,0.647973,0.130088,0.542853,0.117114,0.563140,0.098690,0.608611,0.112447,0.558714,0.074911
BIN:Wisconsin Diagnostic Breast Cancer,0.901959,0.029127,0.906810,0.036541,0.888474,0.021375,0.895669,0.020549,0.903688,0.025423
BIN:Loan House,0.694617,0.035273,0.475461,0.043428,0.460823,0.093157,0.607325,0.044658,0.671143,0.056302
BIN:Heart Failure,0.744983,0.063006,0.580084,0.076828,0.740324,0.054537,0.692413,0.061759,0.755159,0.059507
BIN:Heart Disease,0.735844,0.068672,nan,nan,0.704415,0.058795,0.721702,0.065029,0.670401,0.089768


,GDT roc_auc_mean,GDT roc_auc_std,DNDT roc_auc_mean,DNDT roc_auc_std,GeneticTree roc_auc_mean,GeneticTree roc_auc_std,DL85 roc_auc_mean,DL85 roc_auc_std,sklearn roc_auc_mean,sklearn roc_auc_std
BIN:Blood Transfusion,0.694895,0.060965,0.640991,0.051454,0.620204,0.081754,0.662774,0.040886,0.594091,0.044621
BIN:Banknote Authentication,0.988950,0.005560,0.955636,0.020544,0.949006,0.019105,0.982521,0.006564,0.981197,0.006913
BIN:Titanic,0.802882,0.037348,0.792749,0.039966,0.763943,0.039771,0.743909,0.042171,0.741867,0.040217
BIN:Raisins,0.905148,0.016226,0.863424,0.048260,0.859552,0.022412,0.890429,0.015733,0.817769,0.032838
BIN:Rice,0.975386,0.002617,0.958264,0.004861,0.930088,0.005309,0.961932,0.006599,0.923903,0.013305
BIN:Echocardiogram,0.714017,0.167019,0.579034,0.178860,0.625226,0.097912,0.603888,0.116592,0.592176,0.113532
BIN:Wisconsin Diagnostic Breast Cancer,0.942084,0.023022,0.953729,0.022260,0.896817,0.025874,0.912119,0.036577,0.907265,0.029513
BIN:Loan House,0.699292,0.031545,0.559472,0.075152,0.531611,0.059564,0.579536,0.048554,0.662525,0.062667
BIN:Heart Failure,0.805967,0.048983,0.619994,0.070820,0.740557,0.052719,0.707962,0.095687,0.753062,0.059615
BIN:Heart Disease,0.743213,0.088858,nan,nan,0.749977,0.071921,0.723588,0.071580,0.681382,0.084602


,GDT accuracy_mean,GDT accuracy_std,DNDT accuracy_mean,DNDT accuracy_std,GeneticTree accuracy_mean,GeneticTree accuracy_std,DL85 accuracy_mean,DL85 accuracy_std,sklearn accuracy_mean,sklearn accuracy_std
BIN:Blood Transfusion,0.712081,0.064545,0.700671,0.069176,0.732886,0.048749,0.691275,0.065484,0.696644,0.048749
BIN:Banknote Authentication,0.980292,0.007515,0.886861,0.023426,0.929562,0.020969,0.962774,0.010552,0.981752,0.006321
BIN:Titanic,0.799438,0.030363,0.755056,0.024642,0.776404,0.037920,0.768539,0.028844,0.752247,0.041546
BIN:Raisins,0.850000,0.024721,0.833333,0.024595,0.859444,0.021235,0.849444,0.026580,0.811111,0.027778
BIN:Rice,0.927690,0.006168,0.903150,0.017369,0.927822,0.004980,0.926509,0.008068,0.907218,0.010662
BIN:Echocardiogram,0.738462,0.115128,0.665385,0.075074,0.688462,0.073882,0.703846,0.111538,0.692308,0.059584
BIN:Wisconsin Diagnostic Breast Cancer,0.908850,0.027999,0.914159,0.033593,0.897345,0.021017,0.903540,0.021109,0.910619,0.022889
BIN:Loan House,0.777049,0.029049,0.671311,0.047008,0.701639,0.044323,0.693443,0.050687,0.736885,0.056605
BIN:Heart Failure,0.779661,0.046107,0.616949,0.068303,0.779661,0.041517,0.732203,0.049589,0.777966,0.052761
BIN:Heart Disease,0.741667,0.066771,nan,nan,0.720000,0.060000,0.726667,0.058784,0.673333,0.087305


,GDT total_nodes_mean,GDT total_nodes_std,DNDT total_nodes_mean,DNDT total_nodes_std,GeneticTree total_nodes_mean,GeneticTree total_nodes_std,DL85 total_nodes_mean,DL85 total_nodes_std,sklearn total_nodes_mean,sklearn total_nodes_std
BIN:Blood Transfusion,28.600000,11.926441,24.000000,0.000000,5.400000,3.200000,35.500000,2.418677,246.600000,34.138541
BIN:Banknote Authentication,29.600000,5.444263,24.000000,0.000000,13.400000,3.200000,39.700000,0.900000,47.600000,4.737088
BIN:Titanic,25.800000,6.997142,142.000000,0.000000,9.600000,2.973214,41.800000,1.469694,74.400000,7.432362
BIN:Raisins,39.800000,10.087616,142.000000,0.000000,3.000000,0.000000,43.600000,2.244994,140.000000,11.738824
BIN:Rice,41.400000,7.683749,142.000000,0.000000,3.000000,0.000000,44.800000,1.469694,244.400000,23.273161
BIN:Echocardiogram,19.400000,6.621178,272.000000,0.000000,13.200000,7.613147,44.500000,2.012461,46.200000,8.207314
BIN:Wisconsin Diagnostic Breast Cancer,22.800000,4.853864,1044.000000,0.000000,4.400000,1.800000,43.600000,1.200000,51.400000,4.630335
BIN:Loan House,14.800000,5.688585,2070.000000,0.000000,5.600000,1.280625,41.800000,1.989975,48.200000,10.205881
BIN:Heart Failure,34.200000,8.541663,4120.000000,0.000000,5.200000,5.400000,45.700000,0.900000,65.000000,2.683282
BIN:Heart Disease,28.800000,7.972453,nan,nan,11.000000,4.098780,40.900000,2.343075,40.600000,4.630335


,GDT internal_node_num_mean,GDT internal_node_num_std,DNDT internal_node_num_mean,DNDT internal_node_num_std,GeneticTree internal_node_num_mean,GeneticTree internal_node_num_std,DL85 internal_node_num_mean,DL85 internal_node_num_std,sklearn internal_node_num_mean,sklearn internal_node_num_std
BIN:Blood Transfusion,13.800000,5.963221,8.000000,0.000000,2.200000,1.600000,11.500000,0.806226,122.800000,17.069271
BIN:Banknote Authentication,14.300000,2.722132,8.000000,0.000000,6.200000,1.600000,12.900000,0.300000,23.300000,2.368544
BIN:Titanic,12.400000,3.498571,14.000000,0.000000,4.300000,1.486607,13.600000,0.489898,36.700000,3.716181
BIN:Raisins,19.400000,5.043808,14.000000,0.000000,1.000000,0.000000,14.200000,0.748331,69.500000,5.869412
BIN:Rice,20.200000,3.841875,14.000000,0.000000,1.000000,0.000000,14.600000,0.489898,121.700000,11.636580
BIN:Echocardiogram,9.200000,3.310589,16.000000,0.000000,6.100000,3.806573,14.500000,0.670820,22.600000,4.103657
BIN:Wisconsin Diagnostic Breast Cancer,10.900000,2.426932,20.000000,0.000000,1.700000,0.900000,14.200000,0.400000,25.200000,2.315167
BIN:Loan House,6.900000,2.844293,22.000000,0.000000,2.300000,0.640312,13.600000,0.663325,23.600000,5.102940
BIN:Heart Failure,16.600000,4.270831,24.000000,0.000000,2.100000,2.700000,14.900000,0.300000,32.000000,1.341641
BIN:Heart Disease,13.900000,3.986226,nan,nan,5.000000,2.049390,13.300000,0.781025,19.800000,2.315167


...

,GDT roc_auc_mean,GDT roc_auc_std,DNDT roc_auc_mean,DNDT roc_auc_std,GeneticTree roc_auc_mean,GeneticTree roc_auc_std,DL85 roc_auc_mean,DL85 roc_auc_std,sklearn roc_auc_mean,sklearn roc_auc_std
BIN:Blood Transfusion,0.756376,0.031334,0.675586,0.051609,0.652234,0.100911,0.751233,0.040329,0.962719,0.015958
BIN:Banknote Authentication,0.996096,0.002423,0.960929,0.004786,0.954375,0.014687,0.986617,0.001276,1.000000,0.000000
BIN:Titanic,0.804697,0.054765,0.845702,0.026884,0.929715,0.008739,0.992179,0.001737,0.999185,0.000445
BIN:Raisins,0.918774,0.009013,0.874910,0.032838,0.862614,0.004386,0.919937,0.007077,0.996981,0.002097
BIN:Rice,0.973124,0.002697,0.953564,0.002649,0.923793,0.001668,0.962339,0.006605,0.995713,0.001053
BIN:Echocardiogram,0.879458,0.039753,0.923122,0.054819,0.900037,0.030864,0.961092,0.009769,0.999973,0.000080
BIN:Wisconsin Diagnostic Breast Cancer,0.960487,0.006768,0.977179,0.006539,0.912276,0.013660,0.980044,0.004515,1.000000,0.000000
BIN:Loan House,0.757698,0.059673,0.814250,0.131180,0.888948,0.025569,0.992554,0.002436,0.997500,0.001640
BIN:Heart Failure,0.876675,0.028514,0.942580,0.030290,0.791533,0.044229,0.934762,0.007609,1.000000,0.000000
BIN:Heart Disease,0.847865,0.069438,nan,nan,0.923626,0.018033,0.994280,0.004837,1.000000,0.000000


,GDT accuracy_mean,GDT accuracy_std,DNDT accuracy_mean,DNDT accuracy_std,GeneticTree accuracy_mean,GeneticTree accuracy_std,DL85 accuracy_mean,DL85 accuracy_std,sklearn accuracy_mean,sklearn accuracy_std
BIN:Blood Transfusion,0.731657,0.037801,0.620525,0.100218,0.715318,0.029662,0.755674,0.026637,0.892171,0.016817
BIN:Banknote Authentication,0.989684,0.004418,0.900121,0.019104,0.936339,0.013911,0.967395,0.002471,1.000000,0.000000
BIN:Titanic,0.796636,0.076589,0.791215,0.026275,0.927910,0.008853,0.971809,0.005484,0.986957,0.004115
BIN:Raisins,0.872778,0.011859,0.847407,0.019070,0.862639,0.004279,0.888889,0.004025,0.978750,0.009380
BIN:Rice,0.926859,0.004167,0.895057,0.015950,0.921457,0.002081,0.931398,0.001904,0.969259,0.003558
BIN:Echocardiogram,0.866719,0.042879,0.884809,0.070539,0.879332,0.026136,0.935793,0.013343,0.999342,0.001974
BIN:Wisconsin Diagnostic Breast Cancer,0.937318,0.007850,0.949271,0.012105,0.916447,0.008405,0.966667,0.004138,1.000000,0.000000
BIN:Loan House,0.810811,0.032969,0.823243,0.094279,0.918496,0.013465,0.973780,0.005178,0.983537,0.008102
BIN:Heart Failure,0.876243,0.023853,0.907182,0.048782,0.849167,0.016853,0.927083,0.007739,1.000000,0.000000
BIN:Heart Disease,0.811475,0.069648,nan,nan,0.902881,0.020427,0.990535,0.004887,1.000000,0.000000


,GDT total_nodes_mean,GDT total_nodes_std,DNDT total_nodes_mean,DNDT total_nodes_std,GeneticTree total_nodes_mean,GeneticTree total_nodes_std,DL85 total_nodes_mean,DL85 total_nodes_std,sklearn total_nodes_mean,sklearn total_nodes_std
BIN:Blood Transfusion,28.600000,11.926441,24.000000,0.000000,5.400000,3.200000,35.500000,2.418677,246.600000,34.138541
BIN:Banknote Authentication,29.600000,5.444263,24.000000,0.000000,13.400000,3.200000,39.700000,0.900000,47.600000,4.737088
BIN:Titanic,25.800000,6.997142,142.000000,0.000000,9.600000,2.973214,41.800000,1.469694,74.400000,7.432362
BIN:Raisins,39.800000,10.087616,142.000000,0.000000,3.000000,0.000000,43.600000,2.244994,140.000000,11.738824
BIN:Rice,41.400000,7.683749,142.000000,0.000000,3.000000,0.000000,44.800000,1.469694,244.400000,23.273161
BIN:Echocardiogram,19.400000,6.621178,272.000000,0.000000,13.200000,7.613147,44.500000,2.012461,46.200000,8.207314
BIN:Wisconsin Diagnostic Breast Cancer,22.800000,4.853864,1044.000000,0.000000,4.400000,1.800000,43.600000,1.200000,51.400000,4.630335
BIN:Loan House,14.800000,5.688585,2070.000000,0.000000,5.600000,1.280625,41.800000,1.989975,48.200000,10.205881
BIN:Heart Failure,34.200000,8.541663,4120.000000,0.000000,5.200000,5.400000,45.700000,0.900000,65.000000,2.683282
BIN:Heart Disease,28.800000,7.972453,nan,nan,11.000000,4.098780,40.900000,2.343075,40.600000,4.630335


,GDT internal_node_num_mean,GDT internal_node_num_std,DNDT internal_node_num_mean,DNDT internal_node_num_std,GeneticTree internal_node_num_mean,GeneticTree internal_node_num_std,DL85 internal_node_num_mean,DL85 internal_node_num_std,sklearn internal_node_num_mean,sklearn internal_node_num_std
BIN:Blood Transfusion,13.800000,5.963221,8.000000,0.000000,2.200000,1.600000,11.500000,0.806226,122.800000,17.069271
BIN:Banknote Authentication,14.300000,2.722132,8.000000,0.000000,6.200000,1.600000,12.900000,0.300000,23.300000,2.368544
BIN:Titanic,12.400000,3.498571,14.000000,0.000000,4.300000,1.486607,13.600000,0.489898,36.700000,3.716181
BIN:Raisins,19.400000,5.043808,14.000000,0.000000,1.000000,0.000000,14.200000,0.748331,69.500000,5.869412
BIN:Rice,20.200000,3.841875,14.000000,0.000000,1.000000,0.000000,14.600000,0.489898,121.700000,11.636580
BIN:Echocardiogram,9.200000,3.310589,16.000000,0.000000,6.100000,3.806573,14.500000,0.670820,22.600000,4.103657
BIN:Wisconsin Diagnostic Breast Cancer,10.900000,2.426932,20.000000,0.000000,1.700000,0.900000,14.200000,0.400000,25.200000,2.315167
BIN:Loan House,6.900000,2.844293,22.000000,0.000000,2.300000,0.640312,13.600000,0.663325,23.600000,5.102940
BIN:Heart Failure,16.600000,4.270831,24.000000,0.000000,2.100000,2.700000,14.900000,0.300000,32.000000,1.341641
BIN:Heart Disease,13.900000,3.986226,nan,nan,5.000000,2.049390,13.300000,0.781025,19.800000,2.315167


,GDT leaf_node_num_mean,GDT leaf_node_num_std,DNDT leaf_node_num_mean,DNDT leaf_node_num_std,GeneticTree leaf_node_num_mean,GeneticTree leaf_node_num_std,DL85 leaf_node_num_mean,DL85 leaf_node_num_std,sklearn leaf_node_num_mean,sklearn leaf_node_num_std
BIN:Blood Transfusion,14.800000,5.963221,16.000000,0.000000,3.200000,1.600000,24.000000,1.612452,123.800000,17.069271
BIN:Banknote Authentication,15.300000,2.722132,16.000000,0.000000,7.200000,1.600000,26.800000,0.600000,24.300000,2.368544
BIN:Titanic,13.400000,3.498571,128.000000,0.000000,5.300000,1.486607,28.200000,0.979796,37.700000,3.716181
BIN:Raisins,20.400000,5.043808,128.000000,0.000000,2.000000,0.000000,29.400000,1.496663,70.500000,5.869412
BIN:Rice,21.200000,3.841875,128.000000,0.000000,2.000000,0.000000,30.200000,0.979796,122.700000,11.636580
BIN:Echocardiogram,10.200000,3.310589,256.000000,0.000000,7.100000,3.806573,30.000000,1.341641,23.600000,4.103657
BIN:Wisconsin Diagnostic Breast Cancer,11.900000,2.426932,1024.000000,0.000000,2.700000,0.900000,29.400000,0.800000,26.200000,2.315167
BIN:Loan House,7.900000,2.844293,2048.000000,0.000000,3.300000,0.640312,28.200000,1.326650,24.600000,5.102940
BIN:Heart Failure,17.600000,4.270831,4096.000000,0.000000,3.100000,2.700000,30.800000,0.600000,33.000000,1.341641
BIN:Heart Disease,14.900000,3.986226,nan,nan,6.000000,2.049390,27.600000,1.562050,20.800000,2.315167


In [7]:
from numba import cuda 
if config['computation']['use_gpu']:
    device = cuda.get_current_device()
    device.reset()